In [3]:
import pandas as pd

In [7]:
# Imports CSVs
election_results_df = pd.read_csv('../processed_data/election_results_dataframe.csv')
gb_polls = pd.read_csv('../processed_data/gb_polls.csv')

In [18]:
# Renames column names
election_results_df.rename(columns={'Year': 'election_year'}, inplace=True)

In [29]:
filtered_gb_polls_df = gb_polls[gb_polls['election_year'] <= 2019]


In [30]:
# Merges dataframes
gb_polls_actuals = filtered_gb_polls_df.merge(election_results_df, on='election_year', how='left')

In [31]:
gb_polls_actuals.columns

Index(['startdate', 'enddate', 'pollster', 'samplesize', 'pollster_rating',
       'next_election_date', 'days_until_next_election', 'BRX', 'CON', 'GRE',
       'LAB', 'LIB', 'NAT', 'OTH', 'PLC', 'SNP', 'UKI', 'election_year',
       'Geography', 'Country', 'BRX_ACTUAL_PERCENTAGE',
       'CON_ACTUAL_PERCENTAGE', 'GRE_ACTUAL_PERCENTAGE',
       'LIB_ACTUAL_PERCENTAGE', 'LABOUR_ACTUAL_PERCENTAGE',
       'PLC_ACTUAL_PERCENTAGE', 'SNP_ACTUAL_PERCENTAGE',
       'UKI_ACTUAL_PERCENTAGE', 'OTH_PERCENTAGE'],
      dtype='object')

In [32]:
rename_columns = {
    'BRX': 'BRX_FC',
    'CON': 'CON_FC',
    'GRE': 'GRE_FC',
    'LAB': 'LAB_FC',
    'LIB': 'LIB_FC',
    'NAT': 'NAT_FC',
    'OTH': 'OTH_FC',
    'PLC': 'PLC_FC',
    'SNP': 'SNP_FC',
    'UKI': 'UKI_FC',
    'BRX_ACTUAL_PERCENTAGE': 'BRX_ACT',
    'CON_ACTUAL_PERCENTAGE': 'CON_ACT',
    'GRE_ACTUAL_PERCENTAGE': 'GRE_ACT',
    'LIB_ACTUAL_PERCENTAGE': 'LIB_ACT',
    'LABOUR_ACTUAL_PERCENTAGE': 'LAB_ACT',
    'PLC_ACTUAL_PERCENTAGE': 'PLC_ACT',
    'SNP_ACTUAL_PERCENTAGE': 'SNP_ACT',
    'UKI_ACTUAL_PERCENTAGE': 'UKI_ACT'
}

In [33]:
gb_polls_actuals.rename(columns=rename_columns, inplace=True)


In [34]:
gb_polls_actuals.drop(columns=['Country', 'samplesize'], inplace=True)

In [36]:
combined_df = gb_polls_actuals

In [38]:
combined_df.rename(columns={'pollster_rating': 'rating'}, inplace=True)

In [40]:
combined_df.rename(columns={'next_election_date': 'next_elec_date'}, inplace=True)

In [42]:
combined_df.rename(columns={'days_until_next_election': 'days_to_elec'}, inplace=True)

In [44]:
combined_df.drop(columns='Geography', inplace=True)  # Drop column

In [47]:
combined_df.drop(columns='election_year', inplace=True)  # Drop column

In [49]:
combined_df.to_csv('../processed_data/combined_dataframe.csv', index=False)

In [50]:
combined_df

,startdate,enddate,pollster,rating,next_elec_date,days_to_elec,BRX_FC,CON_FC,GRE_FC,LAB_FC,...,UKI_FC,BRX_ACT,CON_ACT,GRE_ACT,LIB_ACT,LAB_ACT,PLC_ACT,SNP_ACT,UKI_ACT,OTH_PERCENTAGE
0,2004-01-02,2004-01-04,Populus,NaN,2005-05-05,489,NaN,35.0,NaN,40.0,...,NaN,0.000000,32.359595,0.94909,22.025555,35.187187,0.644030,1.518620,2.232152,5.083771
1,2004-01-16,2004-01-18,ICM,NaN,2005-05-05,475,NaN,34.0,NaN,39.0,...,NaN,0.000000,32.359595,0.94909,22.025555,35.187187,0.644030,1.518620,2.232152,5.083771
2,2004-02-06,2004-02-08,Populus,NaN,2005-05-05,454,NaN,31.0,NaN,36.0,...,NaN,0.000000,32.359595,0.94909,22.025555,35.187187,0.644030,1.518620,2.232152,5.083771
3,2004-02-20,2004-02-22,ICM,NaN,2005-05-05,440,NaN,34.0,NaN,36.0,...,NaN,0.000000,32.359595,0.94909,22.025555,35.187187,0.644030,1.518620,2.232152,5.083771
4,2004-03-05,2004-03-07,Populus,NaN,2005-05-05,426,NaN,34.0,NaN,36.0,...,NaN,0.000000,32.359595,0.94909,22.025555,35.187187,0.644030,1.518620,2.232152,5.083771
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3255,2019-12-09,2019-12-11,IpsosMORI,A-,2019-12-12,3,2.0,44.0,3.0,33.0,...,NaN,2.012681,43.631688,2.61203,11.547741,32.080872,0.478805,3.881238,0.071281,3.683662
3256,2019-12-09,2019-12-11,Kantar,B+,2019-12-12,3,3.0,44.0,3.0,32.0,...,NaN,2.012681,43.631688,2.61203,11.547741,32.080872,0.478805,3.881238,0.071281,3.683662
3257,2019-12-10,2019-12-11,Opinium,A-,2019-12-12,2,2.0,45.0,2.0,33.0,...,NaN,2.012681,43.631688,2.61203,11.547741,32.080872,0.478805,3.881238,0.071281,3.683662
3258,2019-12-10,2019-12-11,Panelbase,A-,2019-12-12,2,4.0,43.0,3.0,34.0,...,NaN,2.012681,43.631688,2.61203,11.547741,32.080872,0.478805,3.881238,0.071281,3.683662
